<a href="https://colab.research.google.com/github/stiwari-ds/data-science-competitions/blob/main/machinehack/subscriber_prediction_hackathon/notebooks/01_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%%capture
!pip install --upgrade xgboost
!pip install --upgrade optuna

In [2]:
import os
import gc
import time
import warnings
import subprocess

gc.enable()
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
np.set_printoptions(precision=4)

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

import xgboost
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

SEED = 23
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [3]:
#remove cell to run future versions
assert optuna.__version__ == '3.0.3', f'Change in Optuna version. Original notebook version: 3.0.3'
assert xgboost.__version__ == '1.6.2', f'Change in XGBoost version. Original notebook version: 1.6.2'

In [4]:
#Check GPU availability
try:
    subprocess.check_output('nvidia-smi')
    HAVE_GPU = True
except Exception:
    HAVE_GPU = False

print(f'GPU available: {HAVE_GPU}')

GPU available: True


In [6]:
DATA_URL = 'https://raw.githubusercontent.com/stiwari-ds/data-science-competitions/main/machinehack/subscriber_prediction_hackathon/data'

train = pd.read_csv(f'{DATA_URL}/processed/train.csv') #processed dataset from notebook 00
test = pd.read_csv(f'{DATA_URL}/processed/test.csv')
sample_sub = pd.read_csv(f'{DATA_URL}/raw/submission.csv')

In [7]:
TARGET = 'y_bool'

In [8]:
features = list(test.columns)
num_features = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
cat_features = [f for f in features if f not in num_features]

In [9]:
original_features = ['age', 'job', 'marital', 'education', 'default', 'balance',
                     'housing', 'loan', 'contact', 'day', 'month', 'duration', 
                     'campaign', 'pdays', 'previous', 'poutcome']

cat_only_features = ['age_bins', 'job_groups', 'marital', 'education', 'default',
                     'balance_bins', 'housing', 'loan', 'contact', 'day_bins', 
                     'month_bins', 'duration_bins', 'campaign_bins', 'pdays_bins',
                     'pdays_bool', 'previous_bins', 'previous_bool', 'poutcome']

In [10]:
train[cat_features] = train[cat_features].astype('category')
test[cat_features] = test[cat_features].astype('category')

# Baseline

In [11]:
%%time
scores = []
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
X, y = train[features], train[TARGET]
for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.loc[val_idx], y.iloc[val_idx]

    model = XGBClassifier(
        objective='binary:logistic',
        tree_method='gpu_hist' if HAVE_GPU else 'hist',
        enable_categorical=HAVE_GPU,
        eval_metric='logloss',
        early_stopping_rounds=100, 
        seed=SEED
    ) 
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=0
    )
    val_preds = model.predict_proba(X_val)[:, 1]
    score = log_loss(y_val, val_preds)
    scores.append(score)
    print(f'Fold #{fold}: ({model.best_iteration} rounds) Logloss = {score:.5f}')
    _ = gc.collect()

print(f'\nAvg Logloss = {np.mean(scores):.5f} +/- {np.std(scores):.5f}\n')

Fold #0: (6 rounds) Logloss = 0.58350
Fold #1: (5 rounds) Logloss = 0.58590
Fold #2: (6 rounds) Logloss = 0.58387
Fold #3: (7 rounds) Logloss = 0.58712
Fold #4: (6 rounds) Logloss = 0.58251

Avg Logloss = 0.58458 +/- 0.00168

CPU times: user 5.21 s, sys: 277 ms, total: 5.49 s
Wall time: 5.06 s


# Hyperparameter tuning

In [20]:
def objective(trial, data, model):

    scores = []
    X, y = data

    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.025, 0.3, step=0.025),
        'max_depth': trial.suggest_int('max_depth', 3, 13),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 15),
        'gamma': trial.suggest_float('gamma', 0, 15, step=0.1), #complexity-control
        'alpha': trial.suggest_float('alpha', 0, 5, step=0.05), #L1-reg
        'lambda': trial.suggest_float('lambda', 1e-3, 1e5, log=True), #L2-reg
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.05),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.05),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 1.0, step=0.05),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.5, 1.0, step=0.05),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 5, step=0.05),
        'max_cat_to_onehot': trial.suggest_categorical('max_cat_to_onehot', [2, 5, 12]) 
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model.set_params(**param_grid)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=0
        )
        val_preds = model.predict_proba(X_val)[:, 1]
        scores.append(log_loss(y_val, val_preds))

    return np.mean(scores)

In [21]:
def tune_params(data, model, n_trials, direction):
    study = optuna.create_study(
        sampler=TPESampler(seed=SEED),
        pruner=HyperbandPruner(),
        direction=direction
    )
    study.optimize(
        func=lambda trial: objective(trial, data, model),
        n_trials=n_trials,
        gc_after_trial=True
    )
    return study

# Cross-validation

In [22]:
def cross_validate_predict(data, model, n_splits=5):
    oof_preds = {}  #out-of-fold predictions on train set
    test_preds = {} #predictions on test set for each fold
    scores = [] #scores on validation set

    X, y, X_test = data
       
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=0
        )

        val_preds = model.predict_proba(X_val)[:, 1]
        test_preds[f'fold{fold}'] = model.predict_proba(X_test)[:, 1]
        oof_preds.update(dict(zip(val_idx, val_preds)))

        score = log_loss(y_val, val_preds)
        scores.append(score)
        print(f'Fold #{fold}: Logloss = {score:.5f}')
        _ = gc.collect()
    print(f'Avg. Logloss = {np.mean(scores):.5f} +/- {np.std(scores):.5f}')
    
    oof_preds = pd.Series(oof_preds).sort_index()
    test_preds = pd.DataFrame.from_dict(test_preds)
    test_preds['mean'] = test_preds.mean(axis=1)

    return oof_preds, test_preds

In [23]:
def run_experiment(data, n_trials=10):
    
    X, y, X_test = data

    base_params = {
        'objective': 'binary:logistic',
        'n_estimators': 10000,
        'booster': 'gbtree',
        'eval_metric': 'logloss',
        'early_stopping_rounds': 100,
        'tree_method': 'gpu_hist' if HAVE_GPU else 'hist',
        'predictor': 'gpu_predictor' if HAVE_GPU else 'cpu_predictor',
        'enable_categorical': HAVE_GPU,
        'verbosity': 0,
        'seed': SEED
    }
    
    model = XGBClassifier(**base_params)
    
    print(f'----------Hyperparameter tuning----------')
    start = time.time()
    study = tune_params(
        data=(X, y),
        model=model,
        n_trials=n_trials, 
        direction='minimize' #metric: logloss -> lower is better
    )
    end = time.time()
    print(f'Best trial: {study.best_trial.number} -> Best value: {study.best_value:.5f}')
    print(f'Best hyperparameters:')
    for k, v in study.best_params.items():
        print(f'{k:15} - {v}')
    print(f'[Time taken: {end - start:.2f}s]\n')
    
    print(f'-----Cross-validation and prediction-----')
    start = time.time()
    model.set_params(**study.best_params)
    oof_preds, test_preds = cross_validate_predict(data, model)
    end = time.time()
    print(f'[Time taken: {end - start:.2f}s]\n')

    return oof_preds, test_preds

**Trial run**

In [24]:
optuna.logging.set_verbosity(optuna.logging.INFO)

In [25]:
%%time
op, tp = run_experiment(
    data=(train[features], train[TARGET], test[features]),
    n_trials=3
)

[I 2022-11-16 03:12:22,627] A new study created in memory with name: no-name-158177d4-abdf-4c69-8216-6c9df3534fb1


----------Hyperparameter tuning----------


[I 2022-11-16 03:12:25,023] Trial 0 finished with value: 0.5799144121351697 and parameters: {'learning_rate': 0.17500000000000002, 'max_depth': 13, 'min_child_weight': 12, 'gamma': 4.2, 'alpha': 1.1, 'lambda': 308.87067834937415, 'subsample': 0.55, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.8, 'colsample_bynode': 0.7, 'scale_pos_weight': 1.0, 'max_cat_to_onehot': 5}. Best is trial 0 with value: 0.5799144121351697.
[I 2022-11-16 03:12:30,398] Trial 1 finished with value: 0.7188981490277109 and parameters: {'learning_rate': 0.2, 'max_depth': 13, 'min_child_weight': 13, 'gamma': 0.9, 'alpha': 1.4500000000000002, 'lambda': 0.20112938596732194, 'subsample': 0.95, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.55, 'colsample_bynode': 0.5, 'scale_pos_weight': 4.800000000000001, 'max_cat_to_onehot': 5}. Best is trial 0 with value: 0.5799144121351697.
[I 2022-11-16 03:12:32,087] Trial 2 finished with value: 0.6055210653600238 and parameters: {'learning_rate': 0.275, 'max_depth': 7, 'min_c

Best trial: 0 -> Best value: 0.57991
Best hyperparameters:
learning_rate   - 0.17500000000000002
max_depth       - 13
min_child_weight - 12
gamma           - 4.2
alpha           - 1.1
lambda          - 308.87067834937415
subsample       - 0.55
colsample_bytree - 0.7
colsample_bylevel - 0.8
colsample_bynode - 0.7
scale_pos_weight - 1.0
max_cat_to_onehot - 5
[Time taken: 9.53s]

-----Cross-validation and prediction-----
Fold #0: Logloss = 0.58045
Fold #1: Logloss = 0.58112
Fold #2: Logloss = 0.57838
Fold #3: Logloss = 0.58047
Fold #4: Logloss = 0.57915
Avg. Logloss = 0.57991 +/- 0.00100
[Time taken: 2.77s]

CPU times: user 13.9 s, sys: 164 ms, total: 14.1 s
Wall time: 12.3 s


In [26]:
optuna.logging.set_verbosity(optuna.logging.ERROR)

# Exp 1

In [27]:
%%time
op1, tp1 = run_experiment(
    data=(train[features], train[TARGET], test[features]),
    n_trials=200
)

----------Hyperparameter tuning----------
Best trial: 0 -> Best value: 0.57991
Best hyperparameters:
learning_rate   - 0.17500000000000002
max_depth       - 13
min_child_weight - 12
gamma           - 4.2
alpha           - 1.1
lambda          - 308.87067834937415
subsample       - 0.55
colsample_bytree - 0.7
colsample_bylevel - 0.8
colsample_bynode - 0.7
scale_pos_weight - 1.0
max_cat_to_onehot - 5
[Time taken: 707.00s]

-----Cross-validation and prediction-----
Fold #0: Logloss = 0.58045
Fold #1: Logloss = 0.58112
Fold #2: Logloss = 0.57838
Fold #3: Logloss = 0.58047
Fold #4: Logloss = 0.57915
Avg. Logloss = 0.57991 +/- 0.00100
[Time taken: 2.69s]

CPU times: user 13min 6s, sys: 9.9 s, total: 13min 15s
Wall time: 11min 49s


# Exp 2

In [28]:
%%time
op2, tp2 = run_experiment(
    data=(train[original_features], train[TARGET], test[original_features]),
    n_trials=200
)

----------Hyperparameter tuning----------
Best trial: 160 -> Best value: 0.57976
Best hyperparameters:
learning_rate   - 0.225
max_depth       - 10
min_child_weight - 2
gamma           - 3.5
alpha           - 2.0
lambda          - 52.66764021515259
subsample       - 0.6
colsample_bytree - 0.8500000000000001
colsample_bylevel - 1.0
colsample_bynode - 0.9
scale_pos_weight - 1.0
max_cat_to_onehot - 2
[Time taken: 425.87s]

-----Cross-validation and prediction-----
Fold #0: Logloss = 0.58008
Fold #1: Logloss = 0.58110
Fold #2: Logloss = 0.57842
Fold #3: Logloss = 0.58035
Fold #4: Logloss = 0.57887
Avg. Logloss = 0.57976 +/- 0.00098
[Time taken: 2.01s]

CPU times: user 8min 15s, sys: 7.54 s, total: 8min 23s
Wall time: 7min 7s


# Exp 3

In [29]:
%%time
op3, tp3 = run_experiment(
    data=(train[cat_only_features], train[TARGET], test[cat_only_features]),
    n_trials=200
)

----------Hyperparameter tuning----------
Best trial: 171 -> Best value: 0.57980
Best hyperparameters:
learning_rate   - 0.17500000000000002
max_depth       - 10
min_child_weight - 12
gamma           - 1.8
alpha           - 0.2
lambda          - 0.04467671902420825
subsample       - 0.5
colsample_bytree - 0.55
colsample_bylevel - 0.95
colsample_bynode - 0.75
scale_pos_weight - 1.0
max_cat_to_onehot - 2
[Time taken: 462.83s]

-----Cross-validation and prediction-----
Fold #0: Logloss = 0.58043
Fold #1: Logloss = 0.58104
Fold #2: Logloss = 0.57825
Fold #3: Logloss = 0.58030
Fold #4: Logloss = 0.57897
Avg. Logloss = 0.57980 +/- 0.00103
[Time taken: 2.30s]

CPU times: user 8min 57s, sys: 7.95 s, total: 9min 5s
Wall time: 7min 45s


# Submission files

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
NOTEBOOK = '01'
SUBMISSION_PATH = f'/content/drive/MyDrive/data_science_competitions/machinehack/subscriber_prediction_hackathon/submissions/nb_{NOTEBOOK}'
if not os.path.isdir(SUBMISSION_PATH):
    os.makedirs(SUBMISSION_PATH)

In [32]:
def create_submission_files(test_preds: pd.DataFrame, expt_num: int):
    for col in (test_preds.columns):
        sub = sample_sub.copy()
        sub[TARGET] = test_preds[col]
        sub.to_csv(f'{SUBMISSION_PATH}/{expt_num}_{col}.csv', index=False)

In [33]:
create_submission_files(tp1, '01')
create_submission_files(tp2, '02')
create_submission_files(tp3, '03')